In [1]:
import numpy as np
import numpy.random
import tensorflow as tf
import os
import librosa
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

data_dir = r"C:\Users\Scrap\OneDrive\Documents\Homework\CSCE585\CVDisasterRescue\sound_classification\data\model_data"

HEIGHT = 128
WIDTH = 128
BATCH_SIZE = 32

class_labels = np.array(['human_sounds', 'nonhuman_sounds'])
'''
def load_data(data_dir):
    classes = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x))]
    class_labels ={
        'human_sounds': 1,
        'nonhuman_sounds': 0}
    examples = []
    for cla in classes:
        for file in os.listdir(os.path.join(data_dir, cla)):
            examples.append([os.path.join(data_dir, cla, file), class_labels[cla]])
    return np.array(examples)
'''
def load_data(data_dir):
    classes = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x))]
    class_labels ={
        'human_sounds': 1,
        'nonhuman_sounds': 0}
    examples = []
    for cla in classes:
        for file in os.listdir(os.path.join(data_dir, cla)):
            examples.append([os.path.join(data_dir, cla, file), class_labels[cla]])
    return np.array(examples)


def decode_audio(audio_binary):
    y, sr = librosa.load(audio_binary, sr=16000)
    return tf.squeeze(audio, axis=-1)

def get_waveform(filename):
    try:
        waveform, sr = librosa.load(filename, sr=16000)
        return waveform
    except:
        return []

def get_waveform_and_label(filename):
    label = get_label(filename)
    waveform = get_waveform(filename)
    return waveform, label

def get_spectrogram_image(waveform):
    waveform = tf.cast(waveform, tf.float32)
    spectrogram = tf.signal.stft(waveform, frame_length=2048, frame_step=512, fft_length=2048) #two following lines taken from tutorial
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=-1)
    spectrogram = tf.image.resize(spectrogram, [HEIGHT, WIDTH])
    spectrogram = tf.image.grayscale_to_rgb(spectrogram)
    return spectrogram

def preprocess(dataset):
    X = []
    y = dataset[:, -1]
    count = 0
    for row in tqdm(dataset):
        '''
        waveform = get_waveform(row[0])
        if len(waveform) == 0:
            np.delete(dataset, count, axis=0)
        spectrogram = get_spectrogram_image(waveform)
        X.append(spectrogram)
        '''
        
        #sample_rate, samples = wavfile.read(row[0])
        try:
            audio, sr = librosa.load(row[0], sr=16000)
            '''
            if len(audio) < 64000:
                audio = np.pad(audio, 64000-len(audio))
            audio = audio[:64000]
            '''
            audio = librosa.util.fix_length(audio, 64000)
            #frequencies, times, spectrogram = signal.spectrogram(audio, sr)
            spectrogram = get_spectrogram_image(audio)
            X.append(spectrogram)
        except:
            dataset = np.delete(dataset, count, axis=0)


        count += 1
    
    return X, dataset[:, -1]

C:\Users\Scrap\anaconda3\envs\TensorflowCPU\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\Scrap\anaconda3\envs\TensorflowCPU\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
TEST_SIZE =0.2
file_ds = load_data(data_dir)
rows, cols = file_ds.shape
train_ds = file_ds[:int((1-TEST_SIZE)*rows), :]
test_ds = file_ds[int((1-TEST_SIZE)*rows):, :]
Xtrain, y_train = preprocess(train_ds)
X_test, y_test = preprocess(test_ds)


  0%|          | 0/2548 [00:00<?, ?it/s]C:\Users\Scrap\AppData\Local\Temp\ipykernel_7660\302982795.py:88: FutureWarning: Pass size=64000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  audio = librosa.util.fix_length(audio, 64000)
 18%|█▊        | 459/2548 [01:07<03:01, 11.50it/s]C:\Users\Scrap\anaconda3\envs\TensorflowCPU\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
100%|██████████| 637/637 [03:00<00:00,  3.52it/s]


NameError: name 'X_train' is not defined

In [5]:
X_train = np.asarray(Xtrain)
X_test = np.asarray(X_test)

In [6]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [55]:
spectrogram = X[300]
INPUT_SIZE = (spectrogram.shape)

In [7]:
def conv_block(x, filters, stride=(1,1), padding=False):
        x = tf.keras.layers.Conv2D(filters, (3,3), strides=stride)(x)
        x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
        x = tf.keras.layers.ReLU()(x)
        return x

In [8]:
def get_Model(input_shape):
    input_layer = tf.keras.layers.Input(input_shape)
    x = conv_block(input_layer, 128)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model


In [59]:
model = get_Model((128, 128, 3))
model.compile(optimizer=tf.optimizers.Adam(), 
              loss=tf.keras.losses.BinaryCrossentropy(), 
              metrics=['accuracy'])

model.fit(x=X, y=y,
          epochs=50)

Epoch 1/50
100/100 [==============================] - 50s 489ms/step - loss: 17.0890 - accuracy: 0.4972
Epoch 2/50
100/100 [==============================] - 49s 491ms/step - loss: 0.6708 - accuracy: 0.5569
Epoch 3/50
100/100 [==============================] - 49s 492ms/step - loss: 0.6015 - accuracy: 0.6472
Epoch 4/50
100/100 [==============================] - 49s 493ms/step - loss: 0.5609 - accuracy: 0.6824
Epoch 5/50
 28/100 [=======>......................] - ETA: 35s - loss: 0.5441 - accuracy: 0.7054

KeyboardInterrupt: 